In [1]:
!pwd

import sys
import os

# Detect if we are in Colab or a generic cloud environment (like Vast.ai)
is_colab = 'google.colab' in sys.modules 
is_vast = os.path.exists('/root/.vast_container') # Common Vast.ai indicator
is_cloud = is_colab or is_vast

if is_cloud:
  py_url = "https://raw.githubusercontent.com/fanaustinca/so101ai/refs/heads/main/lerobot_util.py"

  !curl -O {py_url}
  if os.path.exists("lerobot_setup.py"):
    print("\nSuccessfully downloaded lerobot_setup.py")
  else:
    print("\nDownload failed.")
else:
  print("Running on local machine. Skipping cloud setup.")

/home/jieyan/lerobot_ws/so101ai
Running on local machine. Skipping cloud setup.


In [2]:
%conda install python-dotenv -y

# Setting lerobot env
from lerobot_util import setup_lerobot_env

setup_lerobot_env()

Channels:
 - conda-forge
Platform: linux-64
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.
Changed directory to /home/jieyan/lerobot_ws/so101ai
lerobot repository already exists.
Changed directory to lerobot
Cleanly patched pyproject.toml and removed legacy constraints.
Installing lerobot dependencies...
Obtaining file:///home/jieyan/lerobot_ws/so101ai/lerobot
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for lerobot (pyproject.toml):

/home/jieyan/miniforge3/envs/lerobot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


login to hf


wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find train_so101_model.ipynb.
wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from WANDB_API_KEY.
wandb: Currently logged in as: fanjieyan (fanjieyan-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


login to wandb


In [ ]:
# Run the following command to identify USB port for arms
from lerobot_util import print_shell_md
import sys

!ffplay /dev/video0
!v4l2-ctl --list-devices

print_shell_md(
  "Run following command to identify USB port for arms.", 
  f"{sys.executable}",
  "src/lerobot/scripts/lerobot_find_port.py"
)

print_shell_md(
  "Run following command to identify camera index.", 
  f"{sys.executable}",
  "src/lerobot/scripts/lerobot_find_cameras.py"
)


In [ ]:
import sys
import json

from lerobot_util import print_shell_md

leader_port = '/dev/ttyACM1'
follower_port = '/dev/ttyACM0'
wrist_camera_index = 2
top_camera_index = 0
hf_namespace = 'fanaustinca'
data_repo = f"{hf_namespace}/fll_training_data"

# Zoom in 130 to focus on board
!v4l2-ctl -d /dev/video{top_camera_index} --set-ctrl=zoom_absolute=130

camera_config = {
  "wrist": {
    "type": "opencv", 
    "index_or_path": wrist_camera_index, 
    "width": 640, 
    "height": 480, 
    "fps": 30
  },
  "top": {
    "type": "opencv", 
    "index_or_path": top_camera_index, 
    "width": 640, 
    "height": 480, 
    "fps": 30
  }
}

camera_config_str = json.dumps(camera_config, separators=(',', ':'))

arguments = [
    "src/lerobot/scripts/lerobot_teleoperate.py",
    "--robot.type=so101_follower",
    f"--robot.port={follower_port}",
    "--teleop.type=so101_leader",
    f"--teleop.port={leader_port}",
    f"--robot.cameras=\'{camera_config_str}\'",
    "--display_data=true"
]

print_shell_md(
  "Run following command to operate leader arm to control follower arm.", 
  f"{sys.executable}",
  *arguments
)

arguments = [
  "src/lerobot/scripts/lerobot_record.py",
  "--robot.type=so101_follower",
  f"--robot.port={follower_port}"
  f"--robot.cameras=\'{camera_config_str}\'",
  "--teleop.type=so101_leader",
  f"--teleop.port={leader_port}"
  f"--dataset.repo_id={data_repo}",
  "--dataset.num_episodes=50",
  "--dataset.single_task=fll_collection",
  "--dataset.push_to_hub=true",
  "--dataset.private=false"
]

print_shell_md(
  "Run following command to collect training data.", 
  f"{sys.executable}",
  *arguments
)
